# Original Function

In [8]:
# import joblib
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import SimilarityMaps
from scipy.spatial.distance import cdist
import numpy as np

"""import glob
import gzip
import bz2"""
import os
#import _pickle as cPickle

import io
import matplotlib.pyplot as plt

# god hates me so in my version of python I cannot supress these damn user warning so I do this nuclear option instead
"""import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn"""


MODEL_DICT = {
    'Overall Toxicity': ['DT_overall_toxicity_model_final.joblib'],
    'First Trimester Toxicity': ['DT_first_trimester_model_final.joblib'],
    'Second Trimester Toxicity': ['DT_second_trimester_model_final.joblib'],
    'Third Trimester Toxicity': ['DT_third_trimester_model_final.joblib'],
}

# lol I'm just like screw code readability sorry
MODEL_DICT_INVERT = {v: key for key, val in MODEL_DICT.items() for v in val}

CLASSIFICATION_DICT = {
    'Overall Toxicity': {
        0: "Non-toxic",
        1: "Toxic"
    },
    'First Trimester Toxicity': {
        0: "Non-toxic",
        1: "Toxic"
    },
    'Second Trimester Toxicity': {
        0: "Non-toxic",
        1: "Toxic"
    },
    'Third Trimester Toxicity': {
        0: "Non-toxic",
        1: "Toxic"
    },
    }


AD_DICT = {
    True: "Inside",
    False: "Outside"
}


def run_prediction(model, smi, calculate_ad=True, threshold=0.5):
    """_summary_

    Args:
        model (_type_): _description_
        model_data (_type_): _description_
        smi (_type_): _description_
        calculate_ad (bool, optional): _description_. Defaults to True.

    Returns:
        _type_: _description_
    """
    fp = np.zeros((2048, 1))
    # TODO sub in your FP function
    _fp = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smi), radius=2, nBits=2048, useFeatures=False)
    DataStructs.ConvertToNumpyArray(_fp, fp)

    pred_proba = model.predict_proba(fp.reshape(1, -1))[:, 1]
    pred = 1 if pred_proba > threshold else 0

    # used to get proba of the inactive class if deemed inactive
    # if pred == 0:
    #     pred_proba = 1-pred_proba

    # if calculate_ad:
    #     ad = model_data["D_cutoff"] > np.min(cdist(model_data['Descriptors'].to_numpy(), fp.reshape(1, -1)))
    #     return pred, pred_proba, ad
    return pred, pred_proba, None


def get_prob_map(model, smi):
    def get_fp(mol, idx):
        fps = np.zeros((2048, 1))
        _fps = SimilarityMaps.GetMorganFingerprint(mol, idx, radius=3, nBits=2048)
        DataStructs.ConvertToNumpyArray(_fps, fps)
        return fps

    def get_proba(fps):
        return float(model.predict_proba(fps.reshape(1, -1))[:, 1])

    mol = Chem.MolFromSmiles(smi)
    fig, _ = SimilarityMaps.GetSimilarityMapForModel(mol, get_fp, get_proba)
    imgdata = io.StringIO()
    fig.savefig(imgdata, format='svg')
    imgdata.seek(0)  # rewind the data
    plt.savefig(imgdata, format="svg", bbox_inches="tight")

    return imgdata.getvalue()


def main(smi, calculate_ad=True, make_prop_img=False, **kwargs):

    values = {}

    for key, val in kwargs.items():
        if key in MODEL_DICT.keys():  # check if this kwarg is for a model
            if val:  # check if model is turned on
                model_file = MODEL_DICT[key][0]  # Get the first (and presumably only) file in the list
                print(model_file)
                model = joblib.load(model_file)  # load model  # load model

                pred, pred_proba, ad = run_prediction(model, smi, calculate_ad=calculate_ad)

                contrib_svg_str = ""
                if make_prop_img:
                    contrib_svg_str = get_prob_map(model, smi)

                values[key] = [pred, pred_proba, ad, contrib_svg_str]

    processed_results = []
    for key, val in values.items():
        processed_results.append([key, CLASSIFICATION_DICT[key][val[0]], val[1], val[2], val[3]])

    return processed_results


# def write_csv_file(smiles_list, calculate_ad=False):
#     headers = list(MODEL_DICT.keys())
#
#     if calculate_ad:
#         headers = headers + [_ + "_AD" for _ in headers]
#
#     string_file = StringIO()
#     writer = csv.DictWriter(string_file, fieldnames=['SMILES', *headers])
#     writer.writeheader()
#
#     for smiles in tqdm(smiles_list):
#         molecule = MolFromSmiles(smiles)
#
#         row = {'SMILES': smiles}
#
#         if molecule is None:
#             row['SMILES'] = f"(invalid){smiles}"
#             writer.writerow(row)
#             continue
#
#         data = main(smiles, calculate_ad=calculate_ad, **MODEL_DICT)
#
#         for model_name, pred, pred_proba, ad, _ in data:
#             try:
#                 pred_proba = float(pred_proba[:-1]) / 100  # covert back to 0-1 float
#                 row[
#                     model_name] = pred_proba if pred == 1 else 1 - pred_proba  # this is to make sure its proba for class 1
#             except ValueError:
#                 row[model_name] = "No prediction"  # if pred_proba is string skip
#             if calculate_ad:
#                 row[model_name + "_AD"] = ad
#
#         writer.writerow(row)
#
#     return string_file.getvalue()


# if __name__ == "__main__":
#     import argparse
#     import csv
#     from io import StringIO
#     from rdkit.Chem import MolFromSmiles
#     from tqdm import tqdm
#
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--infile", type=str, required=True,
#                         help="location to csv of SMILES")
#     parser.add_argument("--outfile", type=str, default=os.path.join(os.getcwd(), "phakin_output.csv"),
#                         help="location and file name for output")
#     parser.add_argument("--smiles_col", type=str, default="SMILES",
#                         help="column name containing SMILES of interest"),
#     parser.add_argument("--ad", action="store_true",
#                         help="calculate the AD")


# 2nd Function Test

In [9]:
#
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import SimilarityMaps
from scipy.spatial.distance import cdist
import numpy as np

"""import glob
import gzip
import bz2"""
import os
#import _pickle as cPickle

import io
import matplotlib.pyplot as plt

# god hates me so in my version of python I cannot supress these damn user warning so I do this nuclear option instead
"""import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn"""


import joblib  # Ensure this import is at the beginning of your script

# ... [other imports and code] ...

MODEL_DIR = r'c:\Users\ricar\Documents\GitHub\dev-tox\dev_tox\models'  # Directory where models are stored

MODEL_DICT = {
    'Overall Toxicity': [os.path.join(MODEL_DIR, 'DT_overall_toxicity_model_final.joblib')],
    'First Trimester Toxicity': [os.path.join(MODEL_DIR, 'DT_first_trimester_model_final.joblib')],
    'Second Trimester Toxicity': [os.path.join(MODEL_DIR, 'DT_second_trimester_model_final.joblib')],
    'Third Trimester Toxicity': [os.path.join(MODEL_DIR, 'DT_third_trimester_model_final.joblib')],
}


# lol I'm just like screw code readability sorry
MODEL_DICT_INVERT = {v: key for key, val in MODEL_DICT.items() for v in val}

CLASSIFICATION_DICT = {
    'Overall Toxicity': {
        0: "Non-toxic",
        1: "Toxic"
    },
    'First Trimester Toxicity': {
        0: "Non-toxic",
        1: "Toxic"
    },
    'Second Trimester Toxicity': {
        0: "Non-toxic",
        1: "Toxic"
    },
    'Third Trimester Toxicity': {
        0: "Non-toxic",
        1: "Toxic"
    },
    }


AD_DICT = {
    True: "Inside",
    False: "Outside"
}


def run_prediction(model, smi, calculate_ad=True, threshold=0.5):
    """_summary_

    Args:
        model (_type_): _description_
        model_data (_type_): _description_
        smi (_type_): _description_
        calculate_ad (bool, optional): _description_. Defaults to True.

    Returns:
        _type_: _description_
    """
    fp = np.zeros((2048, 1))
    # TODO sub in your FP function
    _fp = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smi), radius=2, nBits=2048, useFeatures=False)
    DataStructs.ConvertToNumpyArray(_fp, fp)

    pred_proba = model.predict_proba(fp.reshape(1, -1))[:, 1]
    pred = 1 if pred_proba > threshold else 0

    # used to get proba of the inactive class if deemed inactive
    # if pred == 0:
    #     pred_proba = 1-pred_proba

    # if calculate_ad:
    #     ad = model_data["D_cutoff"] > np.min(cdist(model_data['Descriptors'].to_numpy(), fp.reshape(1, -1)))
    #     return pred, pred_proba, ad
    return pred, pred_proba, None


def get_prob_map(model, smi):
    def get_fp(mol, idx):
        fps = np.zeros((2048, 1))
        _fps = SimilarityMaps.GetMorganFingerprint(mol, idx, radius=3, nBits=2048)
        DataStructs.ConvertToNumpyArray(_fps, fps)
        return fps

    def get_proba(fps):
        return float(model.predict_proba(fps.reshape(1, -1))[:, 1])

    mol = Chem.MolFromSmiles(smi)
    fig, _ = SimilarityMaps.GetSimilarityMapForModel(mol, get_fp, get_proba)
    imgdata = io.StringIO()
    fig.savefig(imgdata, format='svg')
    imgdata.seek(0)  # rewind the data
    plt.savefig(imgdata, format="svg", bbox_inches="tight")

    return imgdata.getvalue()



def main(smi, calculate_ad=True, make_prop_img=False, **kwargs):
    values = {}

    for key, val in kwargs.items():
        if key in MODEL_DICT.keys():  # check if this kwarg is for a model
            if val:  # check if model is turned on
                model_file = MODEL_DICT[key][0]  # Get the model file path
                print(f"Loading model from: {model_file}")
                model = joblib.load(model_file)  # load the model

                pred, pred_proba, ad = run_prediction(model, smi, calculate_ad=calculate_ad)

                contrib_svg_str = ""
                if make_prop_img:
                    contrib_svg_str = get_prob_map(model, smi)

                values[key] = [pred, pred_proba, ad, contrib_svg_str]

    processed_results = []
    for key, val in values.items():
        processed_results.append([key, CLASSIFICATION_DICT[key][val[0]], val[1], val[2], val[3]])

    return processed_results


# def write_csv_file(smiles_list, calculate_ad=False):
#     headers = list(MODEL_DICT.keys())
#
#     if calculate_ad:
#         headers = headers + [_ + "_AD" for _ in headers]
#
#     string_file = StringIO()
#     writer = csv.DictWriter(string_file, fieldnames=['SMILES', *headers])
#     writer.writeheader()
#
#     for smiles in tqdm(smiles_list):
#         molecule = MolFromSmiles(smiles)
#
#         row = {'SMILES': smiles}
#
#         if molecule is None:
#             row['SMILES'] = f"(invalid){smiles}"
#             writer.writerow(row)
#             continue
#
#         data = main(smiles, calculate_ad=calculate_ad, **MODEL_DICT)
#
#         for model_name, pred, pred_proba, ad, _ in data:
#             try:
#                 pred_proba = float(pred_proba[:-1]) / 100  # covert back to 0-1 float
#                 row[
#                     model_name] = pred_proba if pred == 1 else 1 - pred_proba  # this is to make sure its proba for class 1
#             except ValueError:
#                 row[model_name] = "No prediction"  # if pred_proba is string skip
#             if calculate_ad:
#                 row[model_name + "_AD"] = ad
#
#         writer.writerow(row)
#
#     return string_file.getvalue()


# if __name__ == "__main__":
#     import argparse
#     import csv
#     from io import StringIO
#     from rdkit.Chem import MolFromSmiles
#     from tqdm import tqdm
#
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--infile", type=str, required=True,
#                         help="location to csv of SMILES")
#     parser.add_argument("--outfile", type=str, default=os.path.join(os.getcwd(), "phakin_output.csv"),
#                         help="location and file name for output")
#     parser.add_argument("--smiles_col", type=str, default="SMILES",
#                         help="column name containing SMILES of interest"),
#     parser.add_argument("--ad", action="store_true",
#                         help="calculate the AD")


In [12]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: c:\Users\ricar\Documents\GitHub\dev-tox\dev_tox


In [21]:
smiles_string = "CCO"
results = main(smi=smiles_string, calculate_ad=False, make_prop_img=False, **{"Overall Toxicity": True})
results

Loading model from: c:\Users\ricar\Documents\GitHub\dev-tox\dev_tox\models\DT_overall_toxicity_model_final.joblib


[['Overall Toxicity', 'Non-toxic', array([0.44313292]), None, '']]

In [14]:
model = joblib.load("models/DT_overall_toxicity_model_final.joblib")
print(type(model))  # Check the type of the loaded model


<class 'sklearn.ensemble._forest.RandomForestClassifier'>


# Testing AD Functionality

In [10]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import os
import pickle
from scipy.spatial.distance import pdist

def _get_AD_thresh(fps, file_name):
    dists = pdist(fps)
    mean_1 = dists.mean()
    dists_2 = dists[np.where(dists < mean_1)]
    mean_2 = dists_2.mean()
    std_2 = dists_2.std()

    threshold = mean_2 + (0.5 * std_2)

    pickle.dump((threshold, fps),  open(file_name, "wb"))

def read_sdf_and_save_ad_file(sdf_file, output_file):
    suppl = Chem.SDMolSupplier(sdf_file)
    molecules = [mol for mol in suppl if mol is not None]

    fps = np.array([list(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048, useFeatures=False))
                    for mol in molecules])

    _get_AD_thresh(fps, output_file)




In [5]:
fname1 = "C:/Users/ricar/Documents/GitHub/DT_Modelling/data/DT_overall_tox_sdf_final.sdf"
fname2 = "C:/Users/ricar/Documents/GitHub/DT_Modelling/data/DT_first_trimester_tox_sdf_final.sdf"
fname3 = "C:/Users/ricar/Documents/GitHub/DT_Modelling/data/DT_second_trimester_tox_sdf_final.sdf"
fname4 = "C:/Users/ricar/Documents/GitHub/DT_Modelling/data/DT_third_trimester_tox_sdf_final.sdf"


In [11]:
# Overall FPs
MODEL_DIR = os.path.join(os.path.dirname("c:/Users/ricar/Documents/GitHub/dev-tox/dev_tox"), "models")  # Directory where models are stored
sdf_file = fname1  # Replace with your SDF file path
output_file = os.path.join(MODEL_DIR, 'overall_toxicity_AD.pkl')  # Replace 'your_model_AD.pkl' with your desired output file name

read_sdf_and_save_ad_file(sdf_file, output_file)

In [12]:
# First TM FPs
MODEL_DIR = os.path.join(os.path.dirname("c:/Users/ricar/Documents/GitHub/dev-tox/dev_tox"), "models")  # Directory where models are stored
sdf_file = fname2  # Replace with your SDF file path
output_file = os.path.join(MODEL_DIR, 'first_tri_toxicity_AD.pkl')  # Replace 'your_model_AD.pkl' with your desired output file name

read_sdf_and_save_ad_file(sdf_file, output_file)

In [13]:
# Second TM FPs
MODEL_DIR = os.path.join(os.path.dirname("c:/Users/ricar/Documents/GitHub/dev-tox/dev_tox"), "models")  # Directory where models are stored
sdf_file = fname3  # Replace with your SDF file path
output_file = os.path.join(MODEL_DIR, 'second_tri_toxicity_AD.pkl')  # Replace 'your_model_AD.pkl' with your desired output file name

read_sdf_and_save_ad_file(sdf_file, output_file)

In [14]:
# Third TM FPs
MODEL_DIR = os.path.join(os.path.dirname("c:/Users/ricar/Documents/GitHub/dev-tox/dev_tox"), "models")  # Directory where models are stored
sdf_file = fname4  # Replace with your SDF file path
output_file = os.path.join(MODEL_DIR, 'third_tri_toxicity_AD.pkl')  # Replace 'your_model_AD.pkl' with your desired output file name

read_sdf_and_save_ad_file(sdf_file, output_file)

In [17]:
import pickle
import os

# Assuming your function 'read_sdf_and_save_ad_file' has already been run and the output file has been created

# Path to the output file
MODEL_DIR = os.path.join(os.path.dirname("c:/Users/ricar/Documents/GitHub/dev-tox/dev_tox"), "models")
output_file = os.path.join(MODEL_DIR, 'third_tri_toxicity_AD.pkl')

# Load the output file
with open(output_file, 'rb') as file:
    data = pickle.load(file)

# Check and print the type of data
print(f"Type of the data in the file: {type(data)}")

# If you want to see more details about the content (like length if it's a list or tuple)
if isinstance(data, (list, tuple)):
    print(f"Length of the data: {len(data)}")
    # To show the type of the first element in the tuple/list
    print(f"Type of the first element: {type(data[0])}")


Type of the data in the file: <class 'tuple'>
Length of the data: 2
Type of the first element: <class 'numpy.float64'>
